# Explore mapping file

In [1]:
import os
import pandas as pd


from pathlib import Path


dataDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240531/data/data_matrix.csv')
mappingDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'saur_cohort_20250113', 'source', 'patient_tube_id_mapping_full.tsv'), sep='\t')

dataDf.shape, mappingDf.shape

((116754, 194), (3117, 18))

In [11]:
studyMappingDf = mappingDf.merge(
    dataDf[['person_id', 'visit_occurrence_id']],
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).drop_duplicates()
studyMappingDf

,tube_code,PATIENT_ID,db_ID,pt_age,pt_gender,EPISODE_ID,hospital_admission,hospital_discharge,hospital_in_last_year,genome_species,species_reported,contig_number,length,tube_code_duplicate_or_old,date_of_collection,ID_number,location_additional_02,collected_from_original,person_id,visit_occurrence_id
0,AH19J025,2268214.0,YL5VR,64,Female,12918522.0,2019-10-16,2019-10-22,yes,Enterobacter chengduensis,Enterobacter cloacae complex,134,5175362,NaN,2019-10-16,19-289-1365,A-7EA;HAEM,Blood,2268214,12918522
6,AH20B011,50056.0,64DXZ,50,Male,13266809.0,2020-02-06,2020-02-14,yes,Pseudomonas aeruginosa,Pseudomonas aeruginosa,102,6481548,NaN,2020-02-07,20-038-2462,A-7EA;Bone Marrow & Transplant,Hickman white lumen,50056,13266809
15,AH19G065,62567.0,WD9YG,55,Female,12648702.0,2019-07-25,2019-08-15,yes,Escherichia coli,Escherichia coli,136,5021867,NaN,2019-07-25,19-206-2633,A-HITH;Urology,Blood Venous,62567,12648702
22,AH21E052,138108.0,96MJA,75,Female,14737781.0,2021-05-12,2021-06-11,no,Escherichia coli,Escherichia coli,80,4965577,NaN,2021-05-22,21-142-2384,Ward AC1,Blood,138108,14737781
34,AH20J009,211314.0,2RPLZ,61,Male,13955099.0,2020-09-26,2021-01-27,yes,Enterococcus faecium,Enterococcus faecium,250,3039264,NaN,2020-10-07,20-281-1658,Ward 7 East;nan,Hickman Catheter,211314,13955099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25108,ALF22C004,2084373.0,8K5QL,44,Female,16158768.0,2022-02-18,2022-10-14,yes,Acinetobacter seifertii,Acinetobacter baumannii complex,137,4076994,NaN,2022-03-03,22-062-3603,A-HITH,Blood,2084373,16158768
25190,ALF22H089,1058066.0,98WPR,55,Male,16759482.0,2022-08-04,2022-09-02,no,Streptococcus pseudopneumoniae,Streptococcus mitis group,35,2009898,NaN,2022-08-18,22-230-2926,Ward 7 East,Hickman Catheter,1058066,16759482
25218,ALF22J054,2552781.0,3J6GE,45,Female,16966609.0,2022-10-04,2022-10-25,no,Streptococcus salivarius,Streptococcus infantarius,66,2264549,NaN,2022-10-11,22-284-0722,Ward 7 East,Hickman white lumen,2552781,16966609
25240,ALF22K143,2638484.0,6FCG2,56,Male,16969999.0,2022-10-05,2022-12-10,no,Streptococcus constellatus,Streptococcus anginosus group,49,1935232,NaN,2022-11-23,22-327-0623,Ward AC2,Blood,2638484,16969999


In [12]:
studyMappingDf.location_additional_02.value_counts().reset_index()

,location_additional_02,count
0,Ward 7 East,149
1,Ward 7 East;nan,73
2,nan;nan,61
3,A-7EA;HAEM,52
4,ICU,45
...,...,...
142,Better at home (D) Disability,1
143,Ward 2F,1
144,Better at Home (A),1
145,Ward 2 Discharge Zone,1


In [13]:
studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')].location_additional_02.value_counts().reset_index()

,location_additional_02,count
0,ICU,45
1,ICU;nan,13
2,ICU;Cardiology - Heart Failure,6
3,ICU;HAEM,4
4,ICU;Bone Marrow & Transplant,4
5,ICU;Lung Transplanatation Service (RES1),3
6,ICU;Cardio Thoracic,3
7,ICU;Lung Transplantation,2
8,ICU;AHPB & General Surgery,2
9,ICU;Infectious Diseases,1


In [14]:
studyMappingDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]

654

In [15]:
studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]

70

In [16]:
round(studyMappingDf[studyMappingDf.location_additional_02.notna() & studyMappingDf.location_additional_02.str.lower().str.contains('icu')][['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]/studyMappingDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape[0]*100, 2)

10.7

In [17]:
import os
import pandas as pd


from pathlib import Path


labelsDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240119', 'final', 'icu_admission_labels.csv'))
labelsDf

,person_id,visit_occurrence_id,ICU_ADMISSION
0,2152832,113016,False
1,2152832,133225,False
2,1548773,145028,False
3,2081586,178895,False
4,2152832,189928,False
...,...,...,...
23169,2196440,18035002,False
23170,2449953,18035869,False
23171,852009,18036769,False
23172,2105227,18039086,False


In [20]:
studyLabelsDf = labelsDf.merge(
    dataDf[['person_id', 'visit_occurrence_id']],
    how='inner',
    on=['person_id', 'visit_occurrence_id']
).drop_duplicates()
studyLabelsDf

,person_id,visit_occurrence_id,ICU_ADMISSION
0,2152832,113016,False
1,2152832,133225,False
2,1548773,145028,False
3,2081586,178895,False
4,2152832,189928,False
...,...,...,...
116744,2196440,18035002,False
116745,2449953,18035869,False
116751,852009,18036769,False
116752,2105227,18039086,False


In [21]:
studyLabelsDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]

23174

In [22]:
studyLabelsDf[studyLabelsDf.ICU_ADMISSION][['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]

364

In [23]:
round(studyLabelsDf[studyLabelsDf.ICU_ADMISSION][['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]/studyLabelsDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape[0]*100, 2)

1.57